1.READ DATA

In [20]:
import pandas as pd                                         #importing pandas for handling dataframe
data = pd.read_csv("spam.csv", encoding = 'ISO-8859-1')      #read data with iso encoding
data.rename(columns = {"v1": "Status","v2": "Text1","Unnamed: 2":"Text2","Unnamed: 3":"Text3","Unnamed: 4":"Text4"}, inplace=True) #rename columns after read data
data                                                         #display data

,Status,Text1,Text2,Text3,Text4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


2.EDA

In [21]:
data.isnull().sum()  #check for null values in data

Status       0
Text1        0
Text2     5522
Text3     5560
Text4     5566
dtype: int64

In [22]:
#fill null values with " " and combine all columns except status column to new column name "Text"
data['Text'] = data['Text1'].fillna(" ").astype(str)+" "+data['Text2'].fillna(" ").astype(str)+" "+data['Text3'].fillna(" ").astype(str)+" "+data['Text4'].fillna(" ").astype(str)
data = data.drop(columns=['Text1','Text2','Text3','Text4'])#after combining the "Text" drop column : 'Text1','Text2','Text3','Text4'
data

,Status,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [23]:
data.isnull().sum()  #check for null values in data

Status    0
Text      0
dtype: int64

In [24]:
data.info()             #to get data information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Status  5572 non-null   object
 1   Text    5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [25]:
data.describe()      #to get data description

,Status,Text
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


3.FEATURE EXTRACTION

In [26]:
from sklearn.preprocessing import LabelEncoder   #importing labelencoder from sklearn
le = LabelEncoder()                              #initialising  labelencoder
data["Status"] = le.fit_transform(data["Status"])  #changing values to numerical labels for status column
data


,Status,Text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [27]:
x = data['Text']      #to save independent variable to variable x
y = data['Status']    #to save dependent variable to variable y

4.MODEL BUILDING - FIT THE TF-IDF VECTORIZER

In [28]:
from sklearn.model_selection import train_test_split          #import train_test_split  from sklearn
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2,random_state = 42)    #split the data such that 20% data for testing and 80% for training
print(x_train.shape)      #check the shape of x_train dataset
print(y_train.shape)      #check the shape of y_test dataset
print(x_test.shape)       #check the shape of x_test dataset
print(y_test.shape)       #check the shape of y_test dataset

(4457,)
(4457,)
(1115,)
(1115,)


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer                   #import TfidfVectorizer  
vectorizer = TfidfVectorizer(stop_words = 'english', max_features = 5000)     #initailise TfidfVectorizer  with 5k features and remove english stopwords
x_train_tfidf = vectorizer.fit_transform(x_train)                      #apply fit transform on x_train and initialise to a variable using vectorizer
x_test_tfidf = vectorizer.transform(x_test)                          #apply fit transform on x_test and initialise to a variable using vectorizer


5.MODEL BUILDING AND EVALUATION - NAIVE BAYES

In [30]:
from sklearn.naive_bayes import MultinomialNB   #importing multinomialNB  from sklearn
nb_model = MultinomialNB()                      #initialising multinomial_NB  
nb_model.fit(x_train_tfidf, y_train)              #fit the multinomailNB  model with train dataset
y_pred_nb = nb_model.predict(x_test_tfidf)       #to predict values for test dataset

In [31]:
from sklearn.metrics import classification_report,accuracy_score #importing metrics from sklearn for classification_report,accuracy_score
print("Naive bayes classifier:")                                  #evaluating multinomial naive bayes models
print( classification_report(y_test , y_pred_nb))                 #to get classification report
print("Accuracy:", accuracy_score(y_test , y_pred_nb))             #to get accuracy score

Naive bayes classifier:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       965
           1       1.00      0.80      0.89       150

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115

Accuracy: 0.9730941704035875


6.MODEL BUILDING AND EVALUATION - LOGISTIC REGRESSION

In [32]:
from sklearn.linear_model import LogisticRegression #importing LogisticRegression from sklearn
lr_model = LogisticRegression()                     #initialising  LogisticRegression 
lr_model.fit(x_train_tfidf, y_train)                 #fit the LogisticRegression  model with train dataset
y_pred_lr = lr_model.predict(x_test_tfidf)          #to predict values for test dataset

In [33]:
from sklearn.metrics import classification_report,accuracy_score #importing metrics from sklearn for classification_report,accuracy_score
print("\nLogisticRegression  classifier:")                                
print( classification_report(y_test , y_pred_nb))                 #to get classification report
print("Accuracy:", accuracy_score(y_test , y_pred_nb))             #to get accuracy score


LogisticRegression  classifier:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       965
           1       1.00      0.80      0.89       150

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115

Accuracy: 0.9730941704035875


In [34]:
# Messages to be tested on
new_messages = [
    "Congratulations! You've won a free vacation to the Bahamas. Call now to claim your prize!",
    "Hey, are we still on for the meeting tomorrow?",
    "You've been selected for a $1000 shopping spree! Click here to claim your gift card!",
    "Can we meet tomorrow?"
]

# Transform the new messages to match the training data format
new_messages_tfidf = vectorizer.transform(new_messages)

# Predict using the Naive Bayes model
predictions_nb = nb_model.predict(new_messages_tfidf)

# Predict using the Logistic Regression model
predictions_lr = lr_model.predict(new_messages_tfidf)

# Convert numerical predictions back to labels (0 = ham, 1 = spam)
label_map = {0: 'ham', 1: 'spam'}

# Print predictions for each message
for i, message in enumerate(new_messages):
    print(f"Message: {message}")
    print(f"\tNaive Bayes Prediction       : {label_map[predictions_nb[i]]}")
    print(f"\tLogistic Regression Prediction: {label_map[predictions_lr[i]]}")
    
    print("=" * 100)


Message: Congratulations! You've won a free vacation to the Bahamas. Call now to claim your prize!
	Naive Bayes Prediction       : spam
	Logistic Regression Prediction: spam
Message: Hey, are we still on for the meeting tomorrow?
	Naive Bayes Prediction       : ham
	Logistic Regression Prediction: ham
Message: You've been selected for a $1000 shopping spree! Click here to claim your gift card!
	Naive Bayes Prediction       : spam
	Logistic Regression Prediction: spam
Message: Can we meet tomorrow?
	Naive Bayes Prediction       : ham
	Logistic Regression Prediction: ham
